# Modele probabilistyczne
## Matematyka w analizie sieci i systemów - Ćwiczenia

# Przykłady

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp

sns.set_theme()

## Gamma

* `concentration = alpha`, `alpha > 0`,  
* `rate = beta`, `beta > 0`, 

In [ ]:
# @title Gamma { run: "auto" }
concentration = 1.34  # @param {type:"slider", min:0.01, max:20, step:0.01}
rate = 0.18  # @param {type:"slider", min:0.01, max:20, step:0.01}
max_plot = 50  # @param {type:"slider", min:1, max:20, step:1}

g = tfp.distributions.Gamma(concentration=concentration, rate=rate)

x = np.linspace(0, max_plot, 200)
y = g.prob(x)
plt.title("mean={:02.2f}, stddev={:02.2f}".format(g.mean().numpy(),
                                                  g.stddev().numpy()))
sns.lineplot(x=x, y=y)

## Dwuwymiarowy rozkład normlany


$${\displaystyle {\boldsymbol {\mu }}={\begin{pmatrix}\mu _{X}\\\mu _{Y}\end{pmatrix}},\quad {\boldsymbol {\Sigma }}={\begin{pmatrix}\sigma _{X}^{2}\quad\rho \sigma _{X}\sigma _{Y}\\\rho \sigma _{X}\sigma _{Y}\quad\sigma _{Y}^{2}\end{pmatrix}}.}$$

In [ ]:
import pandas as pd

# @title MVN 2d { run: "auto" }

mux = 100.0  # @param
muy = 20.0  # @param
sigmax = 2.0  # @param
sigmay = 3.0  # @param
rho = 0.71  # @param {type: "slider", min:-1, max:1, step:0.01}

mu = [mux, muy]
sigma = [sigmax, sigmay]
Sigma = np.diag(sigma) ** 2 + rho * np.prod(sigma) * np.rot90(
    np.diag(np.ones_like(mu)))

mvn = tfp.distributions.MultivariateNormalTriL(
    loc=mu, scale_tril=tf.linalg.cholesky(Sigma)
)

sample = pd.DataFrame(mvn.sample(200).numpy(), columns=["X", "Y"])

g = sns.pairplot(sample)
plt.title(
    r"mvn: $\mu=${}, $\sigma$={}, $\rho$={}".format(np.array(mu),
                                                    np.array(sigma), rho)
)
# g.savefig('mvn.pdf')

## Mieszanki

- Tworzenie nowych rozkładów poprzez łączenie innych
- Rozkład kategoryczny $Z$ o $K$ poziomach
- $K$ dowolnych rozkładów 
- $f(x)=p_1 f(x) + p_2f_2(x)+\ldots +p_Kf_K(x)$
- Parametry: Parametry $Z$ + parametry każdej składowej 
- `tfp.distributions.Mixture`


In [ ]:
# @title Default title text {run:"auto"}

p = 0.52  # @param {type:"slider", min:0, max:1, step:0.01}

tfd = tfp.distributions

cat = tfd.Categorical(probs=[p, 1 - p])
comps = [
    tfd.Normal(loc=1.0, scale=1.0),
    tfd.Normal(loc=5.0, scale=1.0),
]

mix = tfd.Mixture(cat=cat, components=comps)

s = mix.sample(1000)

g = sns.histplot(s, kde=True)
# plt.savefig('mix.pdf')

## Tranformed

In [ ]:
d=tfp.distributions.TransformedDistribution(tfp.distributions.Normal(loc=tf.Variable(0.),
                                                                   scale=tfp.util.TransformedVariable(1., tfp.bijectors.Softplus())),
                                          tfp.bijectors.Sigmoid())
plt.hist(d.sample(1000))

## Kde

In [ ]:
# @title test kde
n = 2000  # @param

norm = tfd.Normal(loc=1.0, scale=1.0)
s = norm.sample(n)

g = sns.histplot(s, kde=True, stat="density", label="KDE")
lim = plt.xlim()
x = np.linspace(*lim, 200)
y = norm.prob(x)
plt.plot(x, y, "C2", label="PDF")
plt.legend()


# Zadania

## Zadanie

Dobrać rozkład do zaobserwowanych danych

1. `[0,0,0,1,,0,1,0]`
2. `[4,0,3,1,,2,1,1]`
3. `[4.5,0,3,1.2,,2.0,1,1.1]`

## Zadanie

Rozważmy rozkład Pareto. Jaka jest jego pamięć ($P(X<t+\tau|X> \tau)$)?

$$\mathrm{cdf}(x)=1-\left(\frac{\beta}{x}\right)^\alpha$$

Wynik porównać z wynikiem dla rozkładu wykładniczego

$$\mathrm{cdf}(x)=1-e^{-\lambda x}$$


# Rozkłady i anomalie

- anomalia «odchylenie od normy»
- Co jest normą ? Odpowiada rachunek prawdopodobieństwa.


## Zadanie

Wylosować 10000 liczb z rozkładu normalnego $N(1,1)$. Ile spełnia warunek:

- $x-\mu<0\sigma$
- $x-\mu>1\sigma$
- $x-\mu<-2\sigma$
- $x-\mu>5\sigma$
- $|x-\mu|>5\sigma$

__Co nam mówi obserwacja liczby -8 ?__

## Zadanie

Jak powtórzyć wyniki dla rozkładu wykładniczego

In [ ]:
from IPython.display import display, Markdown

N = 100_000

mu = 1.0
sigma = 1.0
x = np.random.normal(loc=mu, scale=sigma, size=N)

display(Markdown("#### Normal Distribution"))
print(((x - mu) < 0).sum())
print(((x - mu) > 1).sum())
print(((x - mu) < -2).sum())
print(((x - mu) > 5).sum())
print((np.abs(x - mu) > 5 * sigma).sum())

display(Markdown("#### Exponential Distribution"))
x = np.random.exponential(size=N)
mu = 1.0
sigma = 1.0

print(((x - mu) < 0).sum())
print(((x - mu) > 1).sum())
print(((x - mu) < -2).sum())
print(((x - mu) > 5).sum())
print((np.abs(x - mu) > 5 * sigma).sum())


In [ ]:
from tensorflow_probability.substrates import numpy as tfp

tfd = tfp.distributions
import numpy as np

mu = 0
scale = 1
dist = tfd.Normal(loc=mu, scale=scale)
s = dist.sample(10000)
print((s < 0).sum())
print((np.abs(s) > 5).sum())


In [ ]:
# TODO

## Zadanie:

### twierdzenie graniczne

Narysować rozkład sumy 30 zmiennych losowych o rozkładzie wykładniczym

Jak się zmieni wynik dla rozkładu Pareto?

In [ ]:
# TODO

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

x = np.random.exponential(size=(30, 10000))
y = np.sum(x, axis=0)
sns.histplot(y)
plt.figure()
sns.histplot(x.flatten())

x = np.random.pareto(a=3.1, size=(30, 2000))
y = np.sum(x, axis=0)
plt.figure()
sns.histplot(y)
plt.figure()
sns.histplot(x.flatten());
# plt.yscale('log')

## Zadanie
Rozważ sieć z 100 użytkownikami. Średnio każdy użytkownik wymaga 2 Mbit/s, a odchylenie standardowe zapotrzebowania wynosi 4 Mbit/s. Sieć posiada łącze downlinkowe o przepustowości 280 Mbit/s. Oblicz prawdopodobieństwo, że ktoś nie będzie mógł pobierać danych z wymaganą prędkością.

## Zadanie 
Rozważ sieć z 100 użytkownikami. Każdy użytkownik potrzebuje średnio 2 Mb/s. Odchylenie standardowe zapotrzebowania wynosi 4 Mb/s. Oblicz przepustowość łącza downlinkowego, dla której prawdopodobieństwo, że ktoś nie będzie mógł pobierać z wymaganą prędkością, jest mniejsze niż 0,01.


In [ ]:

import scipy.stats as ss

print(1 - ss.norm.cdf(280, 200, 40))
print(ss.norm.ppf(1 - 0.01, 200, 40))

print(dir(ss))


n = 100
tfp.distributions.Normal(loc=n*2.,scale=tf.math.sqrt(n*4.**2)).survival_function(280)

n = 100
tfp.distributions.Normal(loc=n*2.,scale=tf.math.sqrt(n*4.**2)).quantile(0.99)

## Zadanie

$n=100$ klientów, każdy geruje ruch z rozkładem `LogNormal(1.9, 2.2) [Mb/s]`
wyznaczyć

- Pasmo $c$ dla $P(X>c) < 0.01$
- $P(X>10 Gb/s)$